In [35]:
from graphutils import draw_graph_graphical_props, flat_graph_formulation, Node, VAR, COMP, SOLVER, merge_edges, sources
from graphutils import nested_sources, all_variables, edges_to_Ein_Eout
from nestedgraph import build_typedgraph
from compute import Var
from api import Model, adda, addf, addsolver, setsolvefor, addobj, addineq
from execution import edges_from_components, generate_components_and_residuals, Component
from operators import sort_scc, reorder_merge_solve
from nestedgraph import build_typedgraph
from workflow import get_f, implicit_comp_name, SOLVE, OPT, EQ, NEQ, OBJ, IMPL, EXPL, order_from_tree, ENDCOMP, default_solver_options
from workflow import order_from_tree, default_solver_options, mdao_workflow, namefromid
from workflow_mdao import mdao_workflow_with_args
from assembly import architecture_mappings, buildidpvars, build_archi
from runpipeline import nestedform_to_mdao

In [2]:
import openmdao.api as om

In [3]:
import autograd.numpy as anp

In [4]:
from sympy import exp

In [5]:
z1,z2,x,y2 = Var('z1'), Var('z2'), Var('x'), Var('y2')

In [76]:
model = Model(root=OPT)
m = model.root
a = adda(m, 'a', z2+x-0.2*y2)
y1 = adda(m, 'y1', z1**2+a)
adda(m, y2, y1**0.5+z1+z2)
addf(m, x**2+z2+y1+exp(-y2)) #addobj
addf(m, 3.16-y1) #addineq
addf(m, y2-24) #addineq
setsolvefor(m, [x,z1,z2])

In [77]:
edges = edges_from_components(model.components)
tree = model.Ftree, model.Stree, model.Vtree

In [78]:
nodetyperepr = {VAR: '{}', COMP: 'f_{}', SOLVER: 's_{}'}

In [79]:
solvers_options = {1: {'type': OPT}}
comp_options = {3:OBJ, 4:NEQ, 5:NEQ}
var_options = {'x': [0,10], 'z1': [0,10], 'z2': [0,10]}

In [80]:
prob, mdao_in, groups = nestedform_to_mdao(edges, tree, model.components, solvers_options, comp_options, var_options, nodetyperepr, mdf=True)

In [13]:
namingfunc = namefromid(nodetyperepr)

In [14]:
G = flat_graph_formulation(*edges)
merge_order = sort_scc(G)

In [15]:
nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)

In [16]:
sequence = order_from_tree(ntree[0], ntree[1], nedges[1])

In [17]:
sequence

[(SOLVER, 1, None),
 (SOLVER, 2, 1),
 (ENDCOMP, [0, 1, 2], 2),
 (ENDCOMP, [3, 5, 4], 1)]

In [18]:
solvers_options = default_solver_options(ntree, solvers_options)

In [19]:
wf = mdao_workflow(sequence, solvers_options, comp_options, var_options)

In [20]:
components = generate_components_and_residuals(model.components, nedges)

In [21]:
lookup_f = get_f(components, nedges)

In [22]:
wfmdao = mdao_workflow_with_args(wf, lookup_f, namingfunc)

In [23]:
prob, mdao_in, groups = build_archi(nedges, ntree, wfmdao)

In [24]:
# prob.set_val('x', 1.0)
# prob.set_val('z1', 1.0)
# prob.set_val('z2', 1.0)

In [25]:
#prob.run_model()

In [81]:
prob.run_driver();


s_1.s_2
NL: Newton Converged in 3 iterations

s_1.s_2
NL: Newton Converged in 0 iterations

s_1.s_2
NL: Newton Converged in 3 iterations

s_1.s_2
NL: Newton Converged in 3 iterations

s_1.s_2
NL: Newton Converged in 2 iterations

s_1.s_2
NL: Newton Converged in 2 iterations

s_1.s_2
NL: Newton Converged in 3 iterations

s_1.s_2
NL: Newton Converged in 3 iterations

s_1.s_2
NL: Newton Converged in 2 iterations

s_1.s_2
NL: Newton Converged in 1 iterations
Optimization terminated successfully    (Exit mode 0)
            Current function value: [3.18339395]
            Iterations: 9
            Function evaluations: 9
            Gradient evaluations: 9
Optimization Complete
-----------------------------------


In [38]:
allvars = all_variables(*edges_to_Ein_Eout(edges))

In [40]:
xopt = {key: prob.get_val(namingfunc(key, VAR))[0] for key in allvars}

In [41]:
xopt

{'y2': 3.7552778732648413,
 'y1': 3.1600001890218565,
 'a': -0.7510555746459193,
 'z2': 7.0091761203601424e-12,
 'z1': 1.9776389366281641,
 'x': 3.987175173358892e-14}

In [42]:
[comp.evaldict(xopt) for comp in components[0:6]]

[[-0.7510555746459193],
 [3.1600001890218565],
 [3.755277873264841],
 3.183394138181799,
 -1.8902185638935975e-07,
 -20.24472212673516]

In [43]:
om.n2(prob)

In [30]:
workflow = generate_workflow(lookup_f, nedges, ntree, solvers_options={1: {'type': OPT}}, comp_options={2:OBJ, 3:NEQ, 4:NEQ})

In [31]:
workflow

[(OPT, None, 's_1', {'type': OPT, 'designvars': ('x', 'z1', 'z2')}),
 (SOLVE, 's_1', 's_2', {'type': SOLVE}),
 (IMPL,
  's_2',
  'res_f0_f1',
  [(['x_y2', 'x_z1', 'x_z2', 'x_x', 'x_y1'],
    ['x_y1'],
    <bound method Component.evaldict of (('y2', 'z1', 'z2', 'x', 'y1'), 0, (None,), 'None')>,
    <bound method Component.graddict of (('y2', 'z1', 'z2', 'x', 'y1'), 0, (None,), 'None')>,
    1.0),
   (['x_y1', 'x_z1', 'x_z2', 'x_y2'],
    ['x_y2'],
    <bound method Component.evaldict of (('y1', 'z1', 'z2', 'y2'), 1, (None,), 'None')>,
    <bound method Component.graddict of (('y1', 'z1', 'z2', 'y2'), 1, (None,), 'None')>,
    1.0)]),
 (OBJ,
  s_1,
  OBJ,
  (('y1', 'y2', 'z1', 'x'), 2, (None,), 'x**2 + y1 + z1 + exp(-y2)')),
 (NEQ, s_1, NEQ, (('y2',), 4, (None,), 'y2 - 24')),
 (NEQ, s_1, NEQ, (('y1',), 3, (None,), '3.16 - y1'))]